In [ ]:
import os
import requests
from hdbcli import dbapi
from dotenv import load_dotenv
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores.hanavector import HanaDB
from langchain_core.prompts import PromptTemplate

# トークンを取得する関数
def get_token():
    auth_url = os.getenv('AICORE_AUTH_URL')
    client_id = os.getenv('AICORE_CLIENT_ID')
    client_secret = os.getenv('AICORE_CLIENT_SECRET')

    token_url = f"{auth_url}/oauth/token"
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    response = requests.post(token_url, data=data)
    response.raise_for_status()

    token = response.json().get('access_token')
    if not token:
        raise ValueError("トークンが取得できませんでした。")
    return token

# データベースに接続する関数
def connect_database(proxy_client):
    try:
        connection = dbapi.connect(
            address=os.getenv("HANA_DB_ADDRESS"),
            port=int(os.getenv("HANA_DB_PORT")),  # ポートは整数に変換
            user=os.getenv("HANA_DB_USER"),
            password=os.getenv("HANA_DB_PASSWORD"),
            autocommit=True,
            sslValidateCertificate=False,
        )
        print("Database connection successful.")

        embeddings = OpenAIEmbeddings(deployment_id=os.getenv('EMBEDDING_DEPLOYMENT_ID'), proxy_client=proxy_client)
        db = HanaDB(
            connection=connection,
            embedding=embeddings,
            table_name="FILE_EMBEDDINGS",
        )
        print("Database embeddings setup successful.")
        return db
    except Exception as e:
        print("Error connecting to the database or setting up embeddings:", e)
        return None

# チェーンを設定する関数
def setup_chain(db, proxy_client):
    llm = ChatOpenAI(deployment_id=os.getenv('LLM_DEPLOYMENT_ID'), proxy_client=proxy_client)
    prompt_template = """
    You are a helpful advisor. Context related to the prompt is provided.
    Please answer it by referring to the chat history, but also referring to the following context.
    ```
    {context}
    ```
    Chat History: {chat_history}
    Question: {question}
    """
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "chat_history", "question"]
    )

    try:
        retriever = db.as_retriever()
        print("Retriever setup successful.")
    except Exception as e:
        print("Error setting up retriever:", e)
        return None

    chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        verbose=True,  # デバッグのためverboseをTrueに設定
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )
    return chain

# チャットを実行する関数
def run_conversational_chat(chain, query, chat_history):
    print("Running query:", query)
    print("Chat History:", chat_history)

    result = chain({"question": query, "chat_history": chat_history})
    print("Chain result:", result)

    chat_history.append((query, result.get("answer", "No valid response found")))
    
    if "context" in result:
        print("Context used:", result["context"])
    else:
        print("No context used")

    return result.get("answer", "No valid response found")

# 実行部分
if __name__ == "__main__":
    load_dotenv()
    token = get_token()
    proxy_client = get_proxy_client('gen-ai-hub', token=token)

    db = connect_database(proxy_client)

    if db:
        query = "Expanded EditionのProduct Specialistについて、少しでも情報があるファイル名を全て教えてください。拡張子はそれぞれなんですか？"
        results = db.similarity_search(query, k=2)
        print("Similarity Search Results:", results)

        for doc in results:
            print("Document Content:", doc.page_content)
            print("Document Metadata:", doc.metadata)

        chain = setup_chain(db, proxy_client)

        if chain:
            chat_history = []
            answer = run_conversational_chat(chain, query, chat_history)
            print("Answer:", answer)
        else:
            print("Failed to set up the chain.")
    else:
        print("Failed to set up the database.")
